In [80]:
import keras
import os as os
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import cv2 

from IPython.display import Image, display
from keras.datasets import cifar10
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Flatten, Dense, concatenate
from keras.models import Model
from keras.optimizers import Adam
from datetime import datetime
from keras.callbacks import TensorBoard
from keras.layers import multiply, add
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from skimage import io, viewer

import keras
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import csv


from sklearn.model_selection import train_test_split
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from Template_LSTM import LSTM_TEMPLATE as LSTM
from Template_ResNet import ResNet_TEMPLATE as ResNet
from keras.callbacks import TensorBoard
from datetime import datetime

In [81]:
train_csv = pd.read_csv('../Data/train.csv')

filenames = ['../Data/train/' + fname for fname in train_csv['id'].tolist()]
labels = train_csv['has_cactus'].tolist()

train=[]
for file_name in filenames:
    img=cv2.imread(file_name)
    img=img.reshape(32*32*3,)
    img=img/255
    train.append(img)

x_train, x_test, y_train, y_test = train_test_split(train,
                                                    labels,
                                                    train_size=0.9)

num_train = len(x_train)
num_val = len(x_test)

x_train = np.array(x_train)
x_train = np.reshape(x_train, (-1, 32, 32, 3))
print(x_train.shape)

x_test = np.array(x_test)
x_test = np.reshape(x_test, (-1, 32, 32, 3))
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

inputs = Input(shape=(32, 32, 3))
print(inputs)

/anaconda2/envs/tensorflowlatest/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(15750, 32, 32, 3)
(1750, 32, 32, 3)
Tensor("input_2:0", shape=(?, 32, 32, 3), dtype=float32)


In [82]:
## Get hyper param
with open('/Users/aargancointepas/Documents/ESGI-4IBD/MachineLearning/CompetitionKaggle/HyperParam/file_test.csv',
          'r') as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
        row = row[0]
        row = row.split(';')
        name_param = row[0]
        name_modele = row[1]
        batch_size = row[2]
        epochs = row[3]
        lera = row[4]
        activation = row[5]
        nb_layer = row[6]
        print(nb_layer)
        nb_filtre = row[7]
        print(nb_filtre)

        ## Modele
        if name_modele == "LSTM":
            print(name_modele + " " + name_param)
            outputs = LSTM(inputs, nb_filtre, nb_layer)
        if name_modele == "ResNet":
            print(name_modele + " " + name_param)
            outputs = ResNet(nb_layer, inputs, activation)

        ## Run model
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=float(lera)),
                      metrics=['accuracy'])

        model.summary()

        # Prepare model model saving directory.
        save_dir = os.path.join(os.getcwd(), 'res_logs')
        date = datetime.today()
        year = date.strftime("%Y")
        month = date.strftime("%m")
        day = date.strftime("%d")
        hour = date.strftime("%H")
        minute = date.strftime("%M")
        model_name = "{}{}{}{}{}{}_es{}_lr{}_bs{}_{}_ly{}_nf{}" \
            .format(name_modele, year, month, day, hour, minute, epochs, lera, batch_size, activation, nb_layer, nb_filtre)
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
        filepath = os.path.join(save_dir, model_name)
        callbacks = TensorBoard(log_dir=filepath)

        model.fit(x_train, y_train,
                  batch_size=int(batch_size),
                  epochs=int(epochs),
                  validation_data=(x_test, y_test),
                  callbacks=[callbacks])


1
16
LSTM ﻿test1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, 32, 32, 16)   64          conv2d_26[0][0]                  
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 32, 32, 16)   2320        batch_normalization_23[0][0]     
____________________________________________________________________________________________

Train on 15750 samples, validate on 1750 samples
Epoch 1/1
13312/15750 [========================>.....] - ETA: 28s - loss: 2.2619 - acc: 0.7818

KeyboardInterrupt: 

In [83]:
from os import listdir
from os.path import isfile, join

filenames_test = ['../Data/test/' + f for f in listdir('../Data/test/') if isfile(join('../Data/test/', f))]
print(len(filenames_test))
test=[]
i=0
for file_name in filenames_test:
    img=cv2.imread(file_name)
    #print("--------------------------------------------------------------------------------------------------"+str(i)+str(img))
    #i=i+1
    img=img.reshape(32*32*3,)
    img=img/255
    test.append(img)
test = np.array(test)
test = np.reshape(test, (-1, 32, 32, 3))
pred1 = model.predict(test)
predf = pred1.argmax(axis=-1)
print(predf)
#np.savetxt("predict.csv", np.c_[filenames_test,predf], delimiter=",")
df = pd.DataFrame({"name1" : filenames_test, "name2" : predf})
df.to_csv("predict.csv", index=False)

4000
[1 1 1 ... 0 0 1]


In [ ]:
a=[]
for i in filenames:
    a.append(i.split("/")[3])
print(a)

In [ ]:
print(filenames_test[622])